In [3]:
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_csv("data/medium.csv")

In [5]:
dataset.head()

,Unnamed: 0,hair?,feathers?,lays eggs?,milk?,airborne?,aquatic?,predator?,toothed?,backbone?,breathes?,venemous?,fins?,catsize?,brown?,bigger than a car?
0,bass,0,0,1,0,0,1,1,1,1,0,0,1,1,0,0
1,chicken,0,1,1,0,1,0,0,0,1,1,0,0,1,1,0
2,elephant,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1
3,giraffe,1,0,0,1,0,0,0,1,1,1,0,0,0,1,1
4,hawk,0,1,1,0,1,0,1,0,1,1,0,0,1,1,0


In [6]:
X = dataset.iloc[:,1:]

In [7]:
y = dataset.iloc[:,0]

In [8]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
classifier = tree.DecisionTreeClassifier()
classifier = classifier.fit(X,y)

In [9]:
col = list(dataset.columns)
feature = col[1:]
feature

['hair?',
 'feathers?',
 'lays eggs?',
 'milk?',
 'airborne?',
 'aquatic?',
 'predator?',
 'toothed?',
 'backbone?',
 'breathes?',
 'venemous?',
 'fins?',
 'catsize?',
 'brown?',
 'bigger than a car?']

In [10]:
print(classifier.tree_.n_leaves)
print(classifier.n_features_in_)
print(classifier.feature_importances_)

13
15
[0.08333333 0.16666667 0.08333333 0.08333333 0.         0.08333333
 0.08333333 0.         0.08333333 0.         0.         0.
 0.08333333 0.16666667 0.08333333]


In [15]:
from sklearn.tree import _tree

def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []

    def recurse(node, path, paths):

        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            #inp = int(input(f"{name}: "))
            
            if 0 <= threshold:
                p1 += [f"({name} <= {np.round(threshold, 3)})"]
                recurse(tree_.children_left[node], p1, paths)
            else:
                p2 += [f"({name} > {np.round(threshold, 3)})"]
                recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]

    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]

    rules = []
    for path in paths:
        rule = "if "

        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0], 3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]}"
        rules += [rule]


    return rules

In [16]:
rules = get_rules(classifier, feature, y)
print(rules)

['if (predator? <= 0.5) and (catsize? <= 0.5) and (brown? <= 0.5) and (hair? <= 0.5) then class: termite']
